# Prepare data for recommendations display 
- Merge movie data with all meta-data for display + filtering
- Calculate weighted average of ratings
    - Primary sort key for non user filter recommendations
    - Secondary sort key for same cosine similarity in user-item and item-item recommendations
- Downcased versions of variables that user may input (actor, diretor for filters and title for item-item)
- Order and rename columns for display - include non-display columns that are needed for setup at the end
- Save as parquet

In [1]:
import pandas as pd
import os
import re
import numpy as np
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import operator
import fastparquet

In [11]:
df = pd.read_parquet('movies_processed.parq')

In [16]:
imdb_movies 

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
1,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0
2,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,420,$ 45000,NaN,NaN,NaN,24.0,3.0
3,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2019,NaN,NaN,NaN,NaN,28.0,14.0
4,tt0002199,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912,1913,"Biography, Drama",60,USA,English,Sidney Olcott,...,"R. Henderson Bland, Percy Dyer, Gene Gauntier,...","An account of the life of Jesus Christ, based ...",5.7,438,NaN,NaN,NaN,NaN,12.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81268,tt9903716,Jessie,Jessie,2019,2019-03-15,"Horror, Thriller",106,India,Telugu,Aswani Kumar V.,...,"Sritha Chandana, Pavani Gangireddy, Abhinav Go...","Set in an abandoned house, the film follows a ...",7.2,219,NaN,NaN,NaN,NaN,21.0,NaN
81269,tt9905412,Ottam,Ottam,2019,2019-03-08,Drama,120,India,Malayalam,Zam,...,"Nandu Anand, Roshan Ullas, Manikandan R. Achar...","Set in Trivandrum, the story of Ottam unfolds ...",7.8,510,INR 4000000,NaN,$ 4791,NaN,NaN,NaN
81270,tt9905462,Pengalila,Pengalila,2019,2019-03-08,Drama,111,India,Malayalam,T.V. Chandran,...,"Lal, Akshara Kishor, Iniya, Narain, Renji Pani...",An unusual bond between a sixty year old Dalit...,8.4,604,INR 10000000,NaN,NaN,NaN,NaN,NaN
81271,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,8.4,369,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def load_data():
    
    ### processed data from recommendation data exploration.ipynb
    # movie attributes
    df = pd.read_parquet('movies_processed.parq')
    # strip year out of title. Match on ( followed by number. () sometimes valid part of title
    df['title'] = df.title_eng.apply(lambda row: re.split('\\([0-9]', row)[0].strip())    
    
    # number of and average ratings by movie
    movie_ratings = pd.read_parquet('movies_ratings.parq')

    ### data to get additional attributes for display
    links = pd.read_csv('data/ml-25m/links.csv')
    imdb_movies = pd.read_csv('data/imdb/IMDb movies.csv')

    ### genome tags for filtering
    tags = pd.read_csv('data/ml-25m/genome-tags.csv')
    relevance = pd.read_csv('data/ml-25m/genome-scores.csv')
    
    return df, movie_ratings, links, imdb_movies, tags, relevance

## Weighted Average Ratings
- \# of ratings * average rating    
    - If just use average rating, many movies only reviewed once or twice. Want highly rated, frequently watched movies   
    - This does weight poorly rated, but frequently watched movies higher place than well rated, infrequently movies
- Uses:
    - Primary sort key for non user filter recommendations
    - Secondary sort key for same cosine similarity in user-item and item-item recommendations

In [3]:
def weighted_avg(movie_ratings, df):
    # calculate weighted average
    movie_ratings['weighted_avg'] = movie_ratings.avg * movie_ratings.cnt

    # merge with df with movie attributes
    # LEFT merge so keep movies with no ratings (weighted avg = 0) -- can still recommend if fit specific filters
    df = pd.merge(df, movie_ratings[['movieId', 'weighted_avg', 'cnt', 'avg']], on = 'movieId', how = 'left')

    # replace nulls to 0 
    # NOT avg: should display missing if missing
    for var in ['weighted_avg', 'cnt']:
        df[var] = np.where(df[var].isnull(), 0, df[var])
        
    return df

## Merge in additional IMDB attributes

In [4]:
def imdb_merge(imdb_movies, links, df):
    # standardize IMDB IDs
    imdb_movies['imdbId'] = imdb_movies.imdb_title_id.str.split('tt').str[1]
    imdb_movies.imdbId = pd.to_numeric(imdb_movies.imdbId)
    
    x = len(df)
    # merge links to identify IMDB movies
    df = pd.merge(df, links[['movieId', 'imdbId']], on = 'movieId')
    # merge specific IMDB attributes
    df = pd.merge(df, imdb_movies[['imdbId', 'description', 'language', 'duration', 'production_company']])
    assert x == len(df)
    
    return df

## Merge in Genome Tags
Limit to tags with > 75% relevant in a movie for filtering          
__Extension__: incorporate relevant score into weighted average or recommendation score when decide what to show

In [5]:
def genome_merge(tags, relevance, df):
    # merge tags and relevance scores
    tags = pd.merge(tags, relevance, on = 'tagId')

    # limit to tags > 75% relevant to a movie 
    # get list of relevant tags per movie
    tags_relevant = tags[tags.relevance > 0.75].groupby('movieId').tag.apply(list).to_frame()
    tags_relevant.columns = ['tags']

    # merge with dataframe
    # LEFT merge because want to keep movie even if doesn't have any tags
    df = pd.merge(df, tags_relevant, left_on = 'movieId', right_index = True, how = 'left')
    # replace missing tags to empty list
    df.tags = df.tags.apply(lambda d: d if isinstance(d, list) else [])
    
    return df

## Downcase User Input Variables so can match user input non case-sensitive  
Keep non-downcased version for displaying   
Filters:
- Actors
- Directors     
    
Item-Item input: 
- Titles

In [6]:
def downcasing(df):
    df['actors_downcased'] = df.actors_lst.apply(lambda row: [i.lower() for i in row])
    df['directors_downcased'] = df.director_lst.apply(lambda row: [i.lower() for i in row])
    df['title_downcased'] = df.title.apply(lambda row: row.lower())
    return df

## Set up DataFrame for Display

In [7]:
def display_dataframe(df):
    
    df_display = df.copy()
    # drop columns
    df_display = df_display.drop(columns = ['imdbId', 'language'])
    # rename + reorder columns
    df_display.columns = ['movieId', 'title_year', 'Year', 'Genres', 'Director(s)', 'Actors', 'Filming Countries', 'Title',
                          'weighted_avg', 'Number of Ratings', 'Average Rating', 'Description', 'Duration (Minutes)', 
                          'Production Company', 'Tags', 'actors_downcased', 'directors_downcased', 'title_downcased',
                          'Language(s)']
    df_display = df_display[['Title', 'Year', 'Description','Duration (Minutes)', 'Genres', 'Actors', 'Director(s)', 
                             'Production Company', 'Filming Countries', 'Language(s)', 'Tags',
                             'Number of Ratings', 'Average Rating', 'weighted_avg', 'actors_downcased', 'directors_downcased',
                            'title_downcased', 'movieId', 'title_year']]

    return df_display

# Main Function

In [8]:
def main():
    
    # load data
    df, movie_ratings, links, imdb_movies, tags, relevance = load_data()
    # calculate weighted average for sorting
    df = weighted_avg(movie_ratings, df)
    # merge in IMDB metadata and tags 
    df = imdb_merge(imdb_movies, links, df)
    df = genome_merge(tags, relevance, df)
    # downcase user input variables so match user input non case-sensitive (keep regular casing for display) 
    df = downcasing(df)

    # language to list
    for var in ['language']:
        df[var + '_lst'] = df[var].str.split(', ')
        df[var + '_lst'] = df[var + '_lst'].apply(lambda d: d if isinstance(d, list) else [])

    # format df for display
    df_display = display_dataframe(df)
    
    # save as parquet
    df_display.to_parquet('recommendation_display.parq', engine = 'fastparquet', compression = 'GZIP')

In [10]:
main()

In [9]:
#df[df.title.str.startswith('(')].title.unique()